In [10]:
import numpy as np
import cv2

import torchvision.models as models
import torchvision.transforms as transforms

import pytorch_grad_cam 
from pytorch_grad_cam.utils.image import show_cam_on_image

In [11]:
# -*- coding: utf-8 -*-
"""
@Time ： 2023/11/6 16:45
@Auth ： Pengcheng Zheng
@File ：AKstripformer_arch.py
@IDE ：PyCharm
@Motto：ABC(Always Be Coding)
"""
import torch
import torch.nn as nn
import math
from basicsr.utils.registry import ARCH_REGISTRY

class SelectK(nn.Module):
    def __init__(self, input_size, inch):
        super(SelectK, self).__init__()
        self.input_size = input_size
        self.inch = inch
        self.layer0 = nn.Conv2d(self.inch, 1, kernel_size=1, stride=1, padding=0)
        self.layernorm0 = nn.LayerNorm([1,self.input_size,self.input_size])
#         self.sig0 = nn.Sigmoid()
        self.fullconnect = nn.Sequential(
            nn.Linear(self.input_size * self.input_size, self.input_size * self.input_size * 3 // 4),
            nn.LayerNorm(self.input_size * self.input_size * 3 // 4),
#             nn.ReLU(inplace=True),
            nn.Linear(self.input_size * self.input_size * 3 // 4, self.input_size * self.input_size),
            nn.LayerNorm(self.input_size * self.input_size),
#             nn.ReLU(inplace=True),
        )
        self.sig1 = nn.Sigmoid()
        self.min_value = (torch.sin(nn.Parameter(torch.rand(1))) / 2)
        self.max_value = (self.min_value + 0.5)

    def selectMap(self, input, min_threadhold, max_threadhold):
        map_0 = torch.zeros_like(input)
        map_1 = torch.zeros_like(input)
        map_2 = torch.zeros_like(input)
        # 将小于最小阈值的元素映射为3
        map_0[input < min_threadhold] = 1
        # 将大于最大阈值的元素映射为7
        map_2[(input > max_threadhold)] = 1
        # 其余元素映射为5
        map_1[(input >= min_threadhold) & (input <= max_threadhold)] = 1
        return (map_0 - input).detach() + input, (map_1 - input).detach() + input, (map_2 - input).detach() + input

    def forward(self, x):
        out = x

        out = self.layer0(out)
        #print("norm前:",out.shape)
        
        out = self.layernorm0(out)
        
#         out = self.sig0(out)
        
        out = out.view(out.shape[0], 1, self.input_size * self.input_size)

        out = self.fullconnect(out)

        out = out.view(out.shape[0], 1, self.input_size, self.input_size)
                            
        out = self.sig1(out)
                            
        return self.selectMap(out, 0.2, 0.8)

class Embeddings(nn.Module):
    def __init__(self):
        super(Embeddings, self).__init__()

        self.activation = nn.LeakyReLU(0.2, True)

        self.en_layer1_1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            self.activation,
        )

        self.selectK1 = SelectK(100, 64)

        # self.en_layer1_2 = nn.Sequential(
        #     nn.Conv2d(64, 64, kernel_size=3, padding=1),
        #     self.activation,
        #     nn.Conv2d(64, 64, kernel_size=3, padding=1))
        self.en_layer1_2_0 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=1, padding=0),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=1, padding=0))
        self.en_layer1_2_1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=3, padding=1))
        self.en_layer1_2_2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=5, padding=2))

        # self.en_layer1_3 = nn.Sequential(
        #     nn.Conv2d(64, 64, kernel_size=3, padding=1),
        #     self.activation,
        #     nn.Conv2d(64, 64, kernel_size=3, padding=1))
        self.en_layer1_3_0 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=1, padding=0),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=1, padding=0))
        self.en_layer1_3_1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=3, padding=1))
        self.en_layer1_3_2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=5, padding=2))

        # self.en_layer1_4 = nn.Sequential(
        #     nn.Conv2d(64, 64, kernel_size=3, padding=1),
        #     self.activation,
        #     nn.Conv2d(64, 64, kernel_size=3, padding=1))

        self.en_layer2_1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            self.activation,
        )

        self.selectK2 = SelectK(50, 128)

        # self.en_layer2_2 = nn.Sequential(
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1),
        #     self.activation,
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1))
        self.en_layer2_2_0 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=1, padding=0),
            self.activation,
            nn.Conv2d(128, 128, kernel_size=1, padding=0))
        self.en_layer2_2_1 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            self.activation,
            nn.Conv2d(128, 128, kernel_size=3, padding=1))
        self.en_layer2_2_2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            self.activation,
            nn.Conv2d(128, 128, kernel_size=5, padding=2))

        # self.en_layer2_3 = nn.Sequential(
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1),
        #     self.activation,
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1))
        self.en_layer2_3_0 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=1, padding=0),
            self.activation)
        self.en_layer2_3_1 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            self.activation)
        self.en_layer2_3_2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            self.activation)

        # self.en_layer2_4 = nn.Sequential(
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1),
        #     self.activation,
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1))


        self.en_layer3_1 = nn.Sequential(
            nn.Conv2d(128, 320, kernel_size=3, stride=2, padding=1),
            self.activation,
        )


    def forward(self, x):
        hx = self.en_layer1_1(x)
       
        map_0, map_1, map_2 = self.selectK1(hx)
        
        hx = self.activation(
            self.en_layer1_2_0(hx) * map_0 + self.en_layer1_2_1(hx) * map_1 + self.en_layer1_2_2(hx) * map_2 + hx)
        hx = self.activation(
            self.en_layer1_3_0(hx) * map_0 + self.en_layer1_3_1(hx) * map_1 + self.en_layer1_3_2(hx) * map_2 + hx)
        residual_1 = hx

        hx = self.en_layer2_1(hx)

        map_0, map_1, map_2 = self.selectK2(hx)
        
        hx = self.activation(
            self.en_layer2_2_0(hx) * map_0 + self.en_layer2_2_1(hx) * map_1 + self.en_layer2_2_2(hx) * map_2 + hx)
        hx = self.activation(
            self.en_layer2_3_0(hx) * map_0 + self.en_layer2_3_1(hx) * map_1 + self.en_layer2_3_2(hx) * map_2 + hx)
        residual_2 = hx

        hx = self.en_layer3_1(hx)
        print("hx",hx.shape)
        return hx, residual_1, residual_2


class Embeddings_output(nn.Module):
    def __init__(self):
        super(Embeddings_output, self).__init__()

        self.activation = nn.LeakyReLU(0.2, True)

        self.de_layer3_1 = nn.Sequential(
            nn.ConvTranspose2d(320, 192, kernel_size=4, stride=2, padding=1),
            self.activation,
        )
        head_num = 3
        dim = 192

        self.de_layer2_2 = nn.Sequential(
            nn.Conv2d(192+128, 192, kernel_size=1, padding=0),
            self.activation,
        )

        self.de_block_1 = Intra_SA(dim, head_num)
        self.de_block_2 = Inter_SA(dim, head_num)
        self.de_block_3 = Intra_SA(dim, head_num)
        self.de_block_4 = Inter_SA(dim, head_num)
        self.de_block_5 = Intra_SA(dim, head_num)
        self.de_block_6 = Inter_SA(dim, head_num)


        self.de_layer2_1 = nn.Sequential(
            nn.ConvTranspose2d(192, 64, kernel_size=4, stride=2, padding=1),
            self.activation,
        )

        self.de_layer1_3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1, padding=0),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=3, padding=1))
        self.de_layer1_2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            self.activation,
            nn.Conv2d(64, 64, kernel_size=3, padding=1))
        self.de_layer1_1 = nn.Sequential(
            nn.Conv2d(64, 3, kernel_size=3, padding=1),
            self.activation
        )

    def forward(self, x, residual_1, residual_2):


        hx = self.de_layer3_1(x)

        hx = self.de_layer2_2(torch.cat((hx, residual_2), dim = 1))
        hx = self.de_block_1(hx)
        hx = self.de_block_2(hx)
        hx = self.de_block_3(hx)
        hx = self.de_block_4(hx)
        hx = self.de_block_5(hx)
        hx = self.de_block_6(hx)
        hx = self.de_layer2_1(hx)

        hx = self.activation(self.de_layer1_3(torch.cat((hx, residual_1), dim = 1)) + hx)
        hx = self.activation(self.de_layer1_2(hx) + hx)
        hx = self.de_layer1_1(hx)

        return hx

class Attention(nn.Module):
    def __init__(self, head_num):
        super(Attention, self).__init__()
        self.num_attention_heads = head_num
        self.softmax = nn.Softmax(dim=-1)

    def transpose_for_scores(self, x):
        B, N, C = x.size()
        attention_head_size = int(C / self.num_attention_heads)
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3).contiguous()

    def forward(self, query_layer, key_layer, value_layer):
        B, N, C = query_layer.size()
        query_layer = self.transpose_for_scores(query_layer)
        key_layer = self.transpose_for_scores(key_layer)
        value_layer = self.transpose_for_scores(value_layer)
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        _, _, _, d = query_layer.size()
        attention_scores = attention_scores / math.sqrt(d)
        attention_probs = self.softmax(attention_scores)
        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (C,)
        attention_out = context_layer.view(*new_context_layer_shape)

        return attention_out


class Mlp_(nn.Module):
    def __init__(self, hidden_size):
        super(Mlp_, self).__init__()
        self.fc1 = nn.Linear(hidden_size, 4*hidden_size)
        self.fc2 = nn.Linear(4*hidden_size, hidden_size)
        self.act_fn = torch.nn.functional.gelu
        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.normal_(self.fc1.bias, std=1e-6)
        nn.init.normal_(self.fc2.bias, std=1e-6)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act_fn(x)
        x = self.fc2(x)
        return x


# CPE (Conditional Positional Embedding)
class PEG(nn.Module):
    def __init__(self, hidden_size):
        super(PEG, self).__init__()
        self.PEG = nn.Conv2d(hidden_size, hidden_size, kernel_size=3, padding=1, groups=hidden_size)

    def forward(self, x):
        x = self.PEG(x) + x
        return x


class Intra_SA(nn.Module):
    def __init__(self, dim, head_num):
        super(Intra_SA, self).__init__()
        self.hidden_size = dim // 2
        self.head_num = head_num
        self.attention_norm = nn.LayerNorm(dim)
        self.conv_input = nn.Conv2d(dim, dim, kernel_size=1, padding=0)
        self.qkv_local_h = nn.Linear(self.hidden_size, self.hidden_size * 3)  # qkv_h
        self.qkv_local_v = nn.Linear(self.hidden_size, self.hidden_size * 3)  # qkv_v
        self.fuse_out = nn.Conv2d(dim, dim, kernel_size=1, padding=0)
        self.ffn_norm = nn.LayerNorm(dim)
        self.ffn = Mlp_(dim)
        self.attn = Attention(head_num=self.head_num)
        self.PEG = PEG(dim)
    def forward(self, x):
        h = x
        B, C, H, W = x.size()

        x = x.view(B, C, H*W).permute(0, 2, 1).contiguous()
        x = self.attention_norm(x).permute(0, 2, 1).contiguous()
        x = x.view(B, C, H, W)

        x_input = torch.chunk(self.conv_input(x), 2, dim=1)
        feature_h = (x_input[0]).permute(0, 2, 3, 1).contiguous()
        feature_h = feature_h.view(B * H, W, C//2)
        feature_v = (x_input[1]).permute(0, 3, 2, 1).contiguous()
        feature_v = feature_v.view(B * W, H, C//2)
        qkv_h = torch.chunk(self.qkv_local_h(feature_h), 3, dim=2)
        qkv_v = torch.chunk(self.qkv_local_v(feature_v), 3, dim=2)
        q_h, k_h, v_h = qkv_h[0], qkv_h[1], qkv_h[2]
        q_v, k_v, v_v = qkv_v[0], qkv_v[1], qkv_v[2]

        if H == W:
            query = torch.cat((q_h, q_v), dim=0)
            key = torch.cat((k_h, k_v), dim=0)
            value = torch.cat((v_h, v_v), dim=0)
            attention_output = self.attn(query, key, value)
            attention_output = torch.chunk(attention_output, 2, dim=0)
            attention_output_h = attention_output[0]
            attention_output_v = attention_output[1]
            attention_output_h = attention_output_h.view(B, H, W, C//2).permute(0, 3, 1, 2).contiguous()
            attention_output_v = attention_output_v.view(B, W, H, C//2).permute(0, 3, 2, 1).contiguous()
            attn_out = self.fuse_out(torch.cat((attention_output_h, attention_output_v), dim=1))
        else:
            attention_output_h = self.attn(q_h, k_h, v_h)
            attention_output_v = self.attn(q_v, k_v, v_v)
            attention_output_h = attention_output_h.view(B, H, W, C//2).permute(0, 3, 1, 2).contiguous()
            attention_output_v = attention_output_v.view(B, W, H, C//2).permute(0, 3, 2, 1).contiguous()
            attn_out = self.fuse_out(torch.cat((attention_output_h, attention_output_v), dim=1))

        x = attn_out + h
        x = x.view(B, C, H*W).permute(0, 2, 1).contiguous()
        h = x
        x = self.ffn_norm(x)
        x = self.ffn(x)
        x = x + h
        x = x.permute(0, 2, 1).contiguous()
        x = x.view(B, C, H, W)

        x = self.PEG(x)

        return x

class Inter_SA(nn.Module):
    def __init__(self,dim, head_num):
        super(Inter_SA, self).__init__()
        self.hidden_size = dim
        self.head_num = head_num
        self.attention_norm = nn.LayerNorm(self.hidden_size)
        self.conv_input = nn.Conv2d(self.hidden_size, self.hidden_size, kernel_size=1, padding=0)
        self.conv_h = nn.Conv2d(self.hidden_size//2, 3 * (self.hidden_size//2), kernel_size=1, padding=0)  # qkv_h
        self.conv_v = nn.Conv2d(self.hidden_size//2, 3 * (self.hidden_size//2), kernel_size=1, padding=0)  # qkv_v
        self.ffn_norm = nn.LayerNorm(self.hidden_size)
        self.ffn = Mlp_(self.hidden_size)
        self.fuse_out = nn.Conv2d(self.hidden_size, self.hidden_size, kernel_size=1, padding=0)
        self.attn = Attention(head_num=self.head_num)
        self.PEG = PEG(dim)

    def forward(self, x):
        h = x
        B, C, H, W = x.size()

        x = x.view(B, C, H*W).permute(0, 2, 1).contiguous()
        x = self.attention_norm(x).permute(0, 2, 1).contiguous()
        x = x.view(B, C, H, W)

        x_input = torch.chunk(self.conv_input(x), 2, dim=1)
        feature_h = torch.chunk(self.conv_h(x_input[0]), 3, dim=1)
        feature_v = torch.chunk(self.conv_v(x_input[1]), 3, dim=1)
        query_h, key_h, value_h = feature_h[0], feature_h[1], feature_h[2]
        query_v, key_v, value_v = feature_v[0], feature_v[1], feature_v[2]

        horizontal_groups = torch.cat((query_h, key_h, value_h), dim=0)
        horizontal_groups = horizontal_groups.permute(0, 2, 1, 3).contiguous()
        horizontal_groups = horizontal_groups.view(3*B, H, -1)
        horizontal_groups = torch.chunk(horizontal_groups, 3, dim=0)
        query_h, key_h, value_h = horizontal_groups[0], horizontal_groups[1], horizontal_groups[2]

        vertical_groups = torch.cat((query_v, key_v, value_v), dim=0)
        vertical_groups = vertical_groups.permute(0, 3, 1, 2).contiguous()
        vertical_groups = vertical_groups.view(3*B, W, -1)
        vertical_groups = torch.chunk(vertical_groups, 3, dim=0)
        query_v, key_v, value_v = vertical_groups[0], vertical_groups[1], vertical_groups[2]


        if H == W:
            query = torch.cat((query_h, query_v), dim=0)
            key = torch.cat((key_h, key_v), dim=0)
            value = torch.cat((value_h, value_v), dim=0)
            attention_output = self.attn(query, key, value)
            attention_output = torch.chunk(attention_output, 2, dim=0)
            attention_output_h = attention_output[0]
            attention_output_v = attention_output[1]
            attention_output_h = attention_output_h.view(B, H, C//2, W).permute(0, 2, 1, 3).contiguous()
            attention_output_v = attention_output_v.view(B, W, C//2, H).permute(0, 2, 3, 1).contiguous()
            attn_out = self.fuse_out(torch.cat((attention_output_h, attention_output_v), dim=1))
        else:
            attention_output_h = self.attn(query_h, key_h, value_h)
            attention_output_v = self.attn(query_v, key_v, value_v)
            attention_output_h = attention_output_h.view(B, H, C//2, W).permute(0, 2, 1, 3).contiguous()
            attention_output_v = attention_output_v.view(B, W, C//2, H).permute(0, 2, 3, 1).contiguous()
            attn_out = self.fuse_out(torch.cat((attention_output_h, attention_output_v), dim=1))

        x = attn_out + h
        x = x.view(B, C, H*W).permute(0, 2, 1).contiguous()
        h = x
        x = self.ffn_norm(x)
        x = self.ffn(x)
        x = x + h
        x = x.permute(0, 2, 1).contiguous()
        x = x.view(B, C, H, W)

        x = self.PEG(x)

        return x

# @ARCH_REGISTRY.register()
class AKstripformer(nn.Module):
    def __init__(self):
        super(AKstripformer, self).__init__()

        self.encoder = Embeddings()
        head_num = 5
        dim = 320
        self.Trans_block_1 = Intra_SA(dim, head_num)
        self.Trans_block_2 = Inter_SA(dim, head_num)
        self.Trans_block_3 = Intra_SA(dim, head_num)
        self.Trans_block_4 = Inter_SA(dim, head_num)
        self.Trans_block_5 = Intra_SA(dim, head_num)
        self.Trans_block_6 = Inter_SA(dim, head_num)
        self.Trans_block_7 = Intra_SA(dim, head_num)
        self.Trans_block_8 = Inter_SA(dim, head_num)
        self.Trans_block_9 = Intra_SA(dim, head_num)
        self.Trans_block_10 = Inter_SA(dim, head_num)
        self.Trans_block_11 = Intra_SA(dim, head_num)
        self.Trans_block_12 = Inter_SA(dim, head_num)
        self.decoder = Embeddings_output()
        self.conv_last = nn.Conv2d(3,12,(3,3),1,1)
        self.pixelShuffle1 = nn.PixelShuffle(2)
        #self.pixelShuffle2 = nn.PixelShuffle(2)


    def forward(self, x):
        #print("最开始：",x.shape)

        hx, residual_1, residual_2 = self.encoder(x)
        hx = self.Trans_block_1(hx)
        hx = self.Trans_block_2(hx)
        hx = self.Trans_block_3(hx)
        hx = self.Trans_block_4(hx)
        hx = self.Trans_block_5(hx)
        hx = self.Trans_block_6(hx)
        hx = self.Trans_block_7(hx)
        hx = self.Trans_block_8(hx)
        hx = self.Trans_block_9(hx)
        hx = self.Trans_block_10(hx)
        hx = self.Trans_block_11(hx)
        hx = self.Trans_block_12(hx)
        hx = self.decoder(hx, residual_1, residual_2)
        hx = hx + x #(4,3,24,24)
        # print("hx",hx.shape)
        hx = self.conv_last(hx)
        # print("after_conv",hx.shape)
        hx = self.pixelShuffle1(hx)
        #hx = self.pixelShuffle2(hx)
        return hx
# if __name__=='__main__':
#    model = AKstripformer()
#    print(model)
#    x= torch.Tensor(1,3,64,64)
#    lr_deblur = model(x)
#    print("deblur:",lr_deblur.shape)
   # print("sr",sr.shape)
# # model = Stripformer()
# # x = torch.Tensor(4,3,24,24)
# # y = model(x)
# # print(y.shape)


model = AKstripformer()
load_para = torch.load(r"C:\\Users\ASUS\Desktop\PAKB\\net_g_200000.pth")
model.load_state_dict(load_para['params'])
model.eval()

AKstripformer(
  (encoder): Embeddings(
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    (en_layer1_1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (selectK1): SelectK(
      (layer0): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
      (layernorm0): LayerNorm((1, 100, 100), eps=1e-05, elementwise_affine=True)
      (fullconnect): Sequential(
        (0): Linear(in_features=10000, out_features=7500, bias=True)
        (1): LayerNorm((7500,), eps=1e-05, elementwise_affine=True)
        (2): Linear(in_features=7500, out_features=10000, bias=True)
        (3): LayerNorm((10000,), eps=1e-05, elementwise_affine=True)
      )
      (sig1): Sigmoid()
    )
    (en_layer1_2_0): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
   

In [45]:
traget_layers = [model.decoder.de_layer1_1[0]]
print(model.decoder.de_layer1_1[0])

Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [46]:
origin_img = cv2.imread(r"C:\\users\ASUS\Desktop/fengg.jpg")
rgb_img = cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB)

# 图片预处理：resize、裁剪、归一化
trans = transforms.Compose([
    transforms.ToTensor()
#     transforms.Resize(224),
#     transforms.CenterCrop(224)
])
crop_img = trans(rgb_img)
net_input = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(crop_img).unsqueeze(0)

# 将裁剪后的Tensor格式的图像转为numpy格式，便于可视化
canvas_img = (crop_img*255).byte().numpy().transpose(1, 2, 0)
canvas_img = cv2.cvtColor(canvas_img, cv2.COLOR_RGB2BGR)

In [47]:
# 实例化cam，得到指定feature map的可视化数据
cam = pytorch_grad_cam.GradCAMPlusPlus(model=model, target_layers=traget_layers, use_cuda=False)
# print(type(net_input))
grayscale_cam = cam(net_input)
grayscale_cam = grayscale_cam[0, :]

# 将feature map与原图叠加并可视化
src_img = np.float32(canvas_img) / 255
visualization_img = show_cam_on_image(src_img, grayscale_cam, use_rgb=False)
cv2.imwrite("deblur.png",visualization_img)
# cv2.imshow('feature map', visualization_img)
# cv2.waitKey(0)

hx torch.Size([1, 320, 25, 25])


True